In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
#sys.path.insert(1, '/home/samanti/Desktop/THESIS_LMT/imps/src')

import numpy as np
import open3d as o3d
from tqdm.notebook import tqdm
from tqdm import tqdm
import torch
from matplotlib import cm
import matplotlib.pyplot as plt
import torch.optim as optim
import pandas as pd
from sklearn.metrics import jaccard_score

from imps.data.scannet import ScanNetScene, CLASS_NAMES
from imps.sqn.model import Randla
from imps.sqn.data_utils import prepare_input

from imps.point_augment.Common import loss_utils

from imps.point_augment.Augmentor.augmentor import Augmentor
from imps.point_augment.Common import loss_utils, point_augment_utils

SCENE_DIR = '/app/mnt/scans_test'

N_POINTS = int(1.5e5)
RESOLUTION = 25
SIGMAS = None
DEVICE = 'cpu'

IGNORED_LABELS = [0]


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_iou(logits, labels):
    preds = np.argmax(logits, axis=-1)
    ious = []
    
    for c in range(len(CLASS_NAMES)):
        iou = jaccard_score((labels==c).astype(int), (preds==c).astype(int), pos_label=1)
        ious.append(iou)
        
    return np.array(ious)

In [3]:
SCENE_DIR_TRAIN ='/app/mnt/scans'

In [4]:
with open('../../data/train_dataset.txt') as f:
    train_data = f.read().splitlines() 

In [5]:
scene_names_train = os.listdir(SCENE_DIR_TRAIN)

In [6]:
extra_scenes = list(set(scene_names_train).difference(train_data))

In [8]:
print(len(test_scenes))

100


In [8]:
index_list = []
for item in test_scenes:    
    if item.endswith(".zip"):        
        index = test_scenes.index(item)
        index_list.append(index)
        
for index in sorted(index_list, reverse=True):
    del test_scenes[index]

In [4]:
test_scenes = ['scene0664_02', 'scene0583_02', 'scene0316_00', 'scene0702_01', 'scene0300_01', 'scene0050_00', 'scene0599_01', 'scene0559_00', 'scene0064_01', 'scene0334_02', 'scene0701_02', 'scene0664_01', 'scene0558_00', 'scene0695_01', 'scene0046_00', 'scene0652_00', 'scene0256_02', 'scene0187_00', 'scene0629_00', 'scene0678_02', 'scene0684_01', 'scene0655_00', 'scene0690_01', 'scene0217_00', 'scene0435_02', 'scene0406_02', 'scene0423_01', 'scene0591_01', 'scene0647_00', 'scene0086_02', 'scene0328_00', 'scene0406_01', 'scene0629_01', 'scene0686_02', 'scene0050_02', 'scene0474_01', 'scene0046_02', 'scene0081_02', 'scene0025_01', 'scene0690_00', 'scene0685_02', 'scene0527_00', 'scene0653_00', 'scene0011_01', 'scene0686_00', 'scene0084_00', 'scene0671_01', 'scene0653_01', 'scene0575_02', 'scene0222_00', 'scene0696_02', 'scene0257_00', 'scene0164_00', 'scene0643_00', 'scene0357_00', 'scene0278_01', 'scene0695_03', 'scene0500_00', 'scene0338_02', 'scene0019_01', 'scene0077_01', 'scene0084_01', 'scene0088_01', 'scene0518_00', 'scene0378_02', 'scene0353_01', 'scene0684_00', 'scene0458_00', 'scene0658_00', 'scene0693_01', 'scene0063_00', 'scene0207_01', 'scene0427_00', 'scene0146_00', 'scene0382_01', 'scene0149_00', 'scene0153_01', 'scene0633_01', 'scene0609_01', 'scene0553_00', 'scene0343_00', 'scene0329_00', 'scene0378_00', 'scene0458_01', 'scene0648_00', 'scene0568_01', 'scene0354_00', 'scene0580_00', 'scene0648_01', 'scene0187_01', 'scene0356_00', 'scene0651_02', 'scene0609_02', 'scene0591_00', 'scene0644_00', 'scene0432_01', 'scene0131_00', 'scene0426_02', 'scene0701_01', 'scene0169_00']

In [15]:
test_scenes_2 = ['scene0552_00', 'scene0406_00', 'scene0412_00', 'scene0700_02', 'scene0559_01', 'scene0100_02', 'scene0700_00', 'scene0329_01', 'scene0011_00', 'scene0704_01', 'scene0256_00', 'scene0686_01', 'scene0351_00', 'scene0329_02', 'scene0693_00', 'scene0064_00', 'scene0598_00', 'scene0355_01', 'scene0574_01', 'scene0607_00', 'scene0616_01', 'scene0496_00', 'scene0164_03', 'scene0462_00', 'scene0196_00', 'scene0608_01', 'scene0088_00', 'scene0353_00', 'scene0678_01', 'scene0430_00', 'scene0100_01', 'scene0488_01', 'scene0000_00.zip', 'scene0598_02', 'scene0552_01', 'scene0231_02', 'scene0307_00', 'scene0356_02', 'scene0356_01', 'scene0474_03', 'scene0086_01', 'scene0608_02', 'scene0389_00', 'scene0251_00', 'scene0414_00', 'scene0593_00', 'scene0578_01', 'scene0207_02', 'scene0435_00', 'scene0277_00', 'scene0307_02', 'scene0494_00', 'scene0645_00', 'scene0580_01', 'scene0704_00', 'scene0618_00', 'scene0334_00', 'scene0377_00', 'scene0131_01', 'scene0025_02', 'scene0256_01', 'scene0030_01', 'scene0599_00', 'scene0550_00', 'scene0423_00', 'scene0647_01', 'scene0697_03', 'scene0695_02', 'scene0696_00', 'scene0095_00', 'scene0300_00', 'scene0144_01', 'scene0699_00', 'scene0549_01', 'scene0553_01', 'scene0314_00', 'scene0088_03', 'scene0696_01', 'scene0575_01', 'scene0474_00', 'scene0084_02', 'scene0081_00', 'scene0307_01', 'scene0558_01', 'scene0670_00', 'scene0633_00', 'scene0426_00', 'scene0591_02', 'scene0695_00', 'scene0595_00', 'scene0609_03', 'scene0697_01', 'scene0203_01', 'scene0430_01', 'scene0025_00', 'scene0701_00', 'scene0678_00', 'scene0030_00', 'scene0426_01', 'scene0606_01', 'scene0277_01']

In [17]:
index_list = []
for item in test_scenes_2:    
    if item.endswith(".zip"):        
        index = test_scenes_2.index(item)
        index_list.append(index)
        
for index in sorted(index_list, reverse=True):
    del test_scenes_2[index]

In [20]:
print(len(test_scenes_2))

100


In [6]:
randla = Randla(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], device=DEVICE, num_class=len(CLASS_NAMES), interpolator='keops')
randla.load_state_dict(torch.load('../../processed/saved_models/Randla_entire_dataset_10_epochs'))
randla.eval()

Randla(
  (fc0): Conv1d(
    (conv): Conv1d(3, 8, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(
      (bn): BatchNorm1d(8, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
    )
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (dilated_res_blocks): ModuleList(
    (0): Dilated_res_block(
      (mlp1): Conv2d(
        (conv): Conv2d(8, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(
          (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
        )
        (activation): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (lfa): Building_block(
        (mlp1): Conv2d(
          (conv): Conv2d(10, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(
            (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
          )
          (activation): LeakyReLU(negative_slope=0.2, inplace=True)
        )
  

In [7]:




mIOU_list = []
for sn in tqdm(test_scenes):
    scene = ScanNetScene(os.path.join(SCENE_DIR_TRAIN, sn))
    voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(RESOLUTION, N_POINTS, SIGMAS)
    surface_points_, center, pts_min, pts_max = point_augment_utils.scaling_point_cloud(surface_points)
    class_counts = []
    for c in range(len(CLASS_NAMES.keys())):
        class_counts.append(np.sum(point_labels == c))
    class_counts = np.array(class_counts)

    for ign in IGNORED_LABELS:
        class_counts[ign] = 0
    class_weights = class_counts / class_counts.sum()
    features = torch.FloatTensor(surface_colors).unsqueeze(0).to(DEVICE)
    xyz = torch.FloatTensor(surface_points_).unsqueeze(0)
    point_labels = torch.LongTensor(point_labels).unsqueeze(0).to(DEVICE)

    input_points, input_neighbors, input_pools, feat_shape = prepare_input(xyz, k=16, num_layers=3, encoder_dims = [8,32,64], sub_sampling_ratio=4, 
                                                               device=DEVICE)
    
    with torch.no_grad():
        all_logits = randla.forward(features, input_points, input_neighbors, input_pools)
        all_logits = all_logits.squeeze().detach().numpy()
        
    logits_randla_augment = scene.colorize_labels(all_logits.argmax(axis=-1))
    with open(f'../../processed/logits_folder/final_logits_randla_{sn}.npy', 'wb') as f:
        np.save(f, logits_randla_augment)
    with open(f'../../processed/surface_points/surface_points_randla_{sn}.npy', 'wb') as f:
        np.save(f, surface_points)
    IOU = get_iou(all_logits, point_labels.detach().squeeze().numpy())
    IOU_weighted = IOU * class_weights
    #print("IOU_weighted is {}".format(IOU_weighted))
    mIOU = np.sum(IOU_weighted)
    print("mIOU given by: {}".format(mIOU))
    mIOU_list.append(mIOU)
    print(mIOU_list)

  0%|                                                                                                                                                                                       | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_d

mIOU given by: 0.2962460688548717
[0.2962460688548717]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.30391794630616453
[0.2962460688548717, 0.30391794630616453]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

...

mIOU given by: 0.3942255688856783
[0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.3910390506826

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.24812665336476292
[0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.391039050682

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.5404031854064268
[0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.3910390506826

In [8]:
from statistics import mean
mean_IOU_performance = mean(mIOU_list)

In [9]:
mean_IOU_performance

0.41496073788053917

In [14]:
print(mIOU_list)

[0.2962460688548717, 0.30391794630616453, 0.6524141724191902, 0.5080124211021342, 0.2546997191227881, 0.3242468249175954, 0.48479004956123334, 0.5467472828283928, 0.25181696633670264, 0.3229743586259389, 0.1806640666147666, 0.31367196910536943, 0.3036725050839316, 0.3747066545714159, 0.3572061277044272, 0.40287405829159245, 0.36737397224146057, 0.6353939891926504, 0.3761727240153292, 0.6351422203946238, 0.560441796027732, 0.5107469284348097, 0.20716565277126062, 0.4451154906483967, 0.4253084700905224, 0.2904131535243195, 0.5909509165572139, 0.31562296796897144, 0.5209770018455523, 0.4975059521240728, 0.21717606360205344, 0.4317293892832681, 0.3238602108257816, 0.6895299081971155, 0.3690091317136916, 0.2874771060219107, 0.2929327783651647, 0.5474389781053786, 0.40384188640793345, 0.2342361781598103, 0.5722761486238237, 0.30507412415321594, 0.4203328130750395, 0.4809527584990411, 0.6434470726429078, 0.5296326167966681, 0.6076567618895583, 0.3910390506826292, 0.609951648290316, 0.29277736

In [10]:
max(mIOU_list)

0.747812781639543

In [11]:
min(mIOU_list)

0.1806640666147666

In [12]:
mIOU_list[26]

0.5909509165572139

In [21]:


mIOU_list = []
for sn in tqdm(test_scenes_2):
    scene = ScanNetScene(os.path.join(SCENE_DIR_TRAIN, sn))
    voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(RESOLUTION, N_POINTS, SIGMAS)
    surface_points_, center, pts_min, pts_max = point_augment_utils.scaling_point_cloud(surface_points)
    class_counts = []
    for c in range(len(CLASS_NAMES.keys())):
        class_counts.append(np.sum(point_labels == c))
    class_counts = np.array(class_counts)

    for ign in IGNORED_LABELS:
        class_counts[ign] = 0
    class_weights = class_counts / class_counts.sum()
    features = torch.FloatTensor(surface_colors).unsqueeze(0).to(DEVICE)
    xyz = torch.FloatTensor(surface_points_).unsqueeze(0)
    point_labels = torch.LongTensor(point_labels).unsqueeze(0).to(DEVICE)

    input_points, input_neighbors, input_pools, feat_shape = prepare_input(xyz, k=16, num_layers=3, encoder_dims = [8,32,64], sub_sampling_ratio=4, 
                                                               device=DEVICE)
    
    with torch.no_grad():
        all_logits = randla.forward(features, input_points, input_neighbors, input_pools)
        all_logits = all_logits.squeeze().detach().numpy()
        
    logits_randla_augment = scene.colorize_labels(all_logits.argmax(axis=-1))
    with open(f'../../processed/logits_folder_2/final_logits_randla_{sn}.npy', 'wb') as f:
        np.save(f, logits_randla_augment)
    with open(f'../../processed/surface_points_2/surface_points_randla_{sn}.npy', 'wb') as f:
        np.save(f, surface_points)
    IOU = get_iou(all_logits, point_labels.detach().squeeze().numpy())
    IOU_weighted = IOU * class_weights
    #print("IOU_weighted is {}".format(IOU_weighted))
    mIOU = np.sum(IOU_weighted)
    print("mIOU given by: {}".format(mIOU))
    mIOU_list.append(mIOU)
    print(mIOU_list)

  0%|                                                                                                                                                                                       | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_d

mIOU given by: 0.4800828259274245
[0.4800828259274245]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.3141256294622326
[0.4800828259274245, 0.3141256294622326]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

...

mIOU given by: 0.41097741115303804
[0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.36455

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.4041420810206367
[0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.364551

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard 

mIOU given by: 0.4566871219049027
[0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.364551

In [22]:
from statistics import mean
mean(mIOU_list)

0.4086737059630299

In [23]:
max(mIOU_list)

0.7067245979891372

In [24]:
min(mIOU_list)

0.18795980837983148

In [25]:
print(mIOU_list)

[0.4800828259274245, 0.3141256294622326, 0.43314472897323886, 0.3108838838691709, 0.4217744661366317, 0.30750138626132656, 0.23392175229936546, 0.5477260303527688, 0.4806466370151126, 0.4273892546729509, 0.23403418107045632, 0.622984223580449, 0.28395898477729614, 0.5053798888534788, 0.36854260904913283, 0.23487570158381382, 0.2776467227756309, 0.6755961181605183, 0.37903712673606926, 0.5504114010316731, 0.37429026290122175, 0.496385323357798, 0.39196129947896174, 0.37924939647851275, 0.5853269475386425, 0.255262318179988, 0.6848665717060205, 0.30138798440552894, 0.5921036056828485, 0.6546765959275835, 0.27826561055984433, 0.22863031747535176, 0.27819431972682734, 0.5282928933268631, 0.29809156976279433, 0.2563069268808263, 0.41665888131670775, 0.4455425174190389, 0.34989910732345086, 0.5253094595573349, 0.2890501560457343, 0.4635251003659016, 0.24949786498487941, 0.3703191870359074, 0.3865878299278053, 0.5965633281022387, 0.29854842588276786, 0.36455184643516014, 0.4672057744440935, 0

In [2]:
scene = ScanNetScene(SCENE_DIR)

voxel_grid, surface_points, surface_colors, vicinity_points, vicinity_distances, point_labels = scene.create_if_data(
    RESOLUTION, N_POINTS, SIGMAS
)

center = surface_points.mean(axis=0, keepdims=True)
surface_points -= center
    
pts_min = surface_points.min(axis=0, keepdims=True)
pts_max = surface_points.max(axis=0, keepdims=True)
surface_points = (surface_points - pts_min) / (pts_max - pts_min)
surface_points -= 0.5

class_counts = []
for c in range(len(CLASS_NAMES.keys())):
    class_counts.append(np.sum(point_labels == c))
class_counts = np.array(class_counts)

for ign in IGNORED_LABELS:
    class_counts[ign] = 0
class_weights = class_counts / class_counts.sum()

In [3]:
# In randla query = input, labels = query_labels
features = torch.FloatTensor(surface_colors).unsqueeze(0).to(DEVICE)
xyz = torch.FloatTensor(surface_points).unsqueeze(0)
point_labels = torch.LongTensor(point_labels).unsqueeze(0).to(DEVICE)

input_points, input_neighbors, input_pools, feat_shape = prepare_input(xyz, k=16, num_layers=3, encoder_dims = [8,32,64], sub_sampling_ratio=4, 
                                                           device=DEVICE)

randla = Randla(d_feature=3, d_in=8, encoder_dims=[8, 32, 64], device=DEVICE, num_class=len(CLASS_NAMES), interpolator='keops')
randla.load_state_dict(torch.load('../../processed/saved_models/Randla_entire_dataset_10_epochs'))
randla.eval()

Randla(
  (fc0): Conv1d(
    (conv): Conv1d(3, 8, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(
      (bn): BatchNorm1d(8, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
    )
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (dilated_res_blocks): ModuleList(
    (0): Dilated_res_block(
      (mlp1): Conv2d(
        (conv): Conv2d(8, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(
          (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
        )
        (activation): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (lfa): Building_block(
        (mlp1): Conv2d(
          (conv): Conv2d(10, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(
            (bn): BatchNorm2d(4, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
          )
          (activation): LeakyReLU(negative_slope=0.2, inplace=True)
        )
  

In [4]:
with torch.no_grad():
    all_logits = randla.forward(features, input_points, input_neighbors, input_pools)
    all_logits = all_logits.squeeze().detach().numpy()

In [5]:
df = pd.DataFrame()
df['class'] = CLASS_NAMES.keys()
df['iou'] = get_iou(all_logits, point_labels.detach().squeeze().numpy())
df['weight'] = class_weights
df['iou_weighted'] = df['iou'] * df['weight']

print("mIOU:", df.iou_weighted.sum())

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


mIOU: 0.2609062717804598
